In [21]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import openpyxl
import re
import io
import os

In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [23]:
def autodict(df, var: str):
    unique = df[var].unique()
    dicionario = dict(zip(unique, range(len(unique))))
    return dicionario

In [24]:
def atrib_lbl(nome_dict, nome_arquivo, nome_csv):
    """
    Escreve um arquivo de atribuição de value labels do SPSS a partir de um dicionário.

    Parâmetros:
    - nome_dict (str): O nome da variável que contém o dicionário de rótulos de valor. Deve seguir o padrão "nome_da_variavel_dict".
    - nome_arquivo (str): O nome do arquivo de saída. Deve incluir a extensão ".sps".
    - nome_csv (str): O nome do arquivo CSV contendo o mapa das variáveis e seus labels.
    """
    # Extrai o nome da variável a partir do nome do dicionário
    nome_variavel_match = re.search(r'^(\w+)_dict$', nome_dict)
    if nome_variavel_match is None:
        print("Erro: nome do dicionário não está no formato esperado 'nome_da_variavel_dict'.")
        return
    
    nome_variavel = nome_variavel_match.group(1)

    # Verifica se o arquivo já existe
    arquivo_existe = os.path.exists(nome_arquivo)

    # Cria um buffer de saída para armazenar o conteúdo do arquivo temporariamente
    buffer_saida = io.StringIO()

    # Abre o arquivo CSV e busca o label correspondente à variável atual
    df = pd.read_csv(nome_csv, encoding='latin1', delimiter=';')
    idv = df.loc[df['variavel'] == nome_variavel, 'idV'].iloc[0]

    # Escreve um comentário com a informação da variável
    buffer_saida.write(f'*****************************************\n')
    buffer_saida.write(f'** Atribuição de value labels para {nome_variavel} - {idv}\n')
    buffer_saida.write(f'*****************************************\n\n')

    # Escrevendo a sintaxe para cada valor no dicionário
    for label, categoria in eval(nome_dict).items():
        buffer_saida.write(f'{categoria} "{label}", ')

    # Remove a última vírgula e adiciona um ponto final para encerrar a declaração de rótulos de valor para a variável atual
    buffer_saida.write(f'\nVARIABLE LABELS {nome_variavel} "{idv}".\n')
    buffer_saida.write(f'EXECUTE.\n\n')

    # Grava o conteúdo do buffer no arquivo com a codificação correta
    with open(nome_arquivo, 'a' if arquivo_existe else 'w', encoding='utf-8') as f:
        f.write(buffer_saida.getvalue())

    print(f'Arquivo "{nome_arquivo}" atualizado com sucesso.')

In [25]:
def sn_dict(df, col_name: str):
    global col_name_dict  # Adicione esta linha para declarar col_name_dict como global
    
    col_name_dict = col_name + '_dict'
    sim_nao_dict = {'Sim': 1, 'Não': 0, 'Não Sabe': 99}
    df[col_name] = df[col_name].map(sim_nao_dict)
    df[col_name] = df[col_name].map(lambda x: int(x) if not np.isnan(x) else -1)
    
    globals()[col_name_dict] = sim_nao_dict  # Adicione esta linha para criar uma variável global com o nome col_name_dict e atribuir sim_nao_dict a ela

    return print('Dicionário sim/não/não_sabe concluído. Frequências:'), df[col_name].value_counts()

In [118]:
dicionario = "./Municipal/Dicionarios/Dicionario_Formulario2.csv"
file = "Municipal\Respostas\Formulario_2.xlsx"
df = pd.read_excel(file, header = 0, index_col=0)
df.drop(['nome_municipio', 'uf', 'versao_salva'], axis = 1, inplace=True)
df

,v8973,v13819,v13820,v13821_10,v13821_15,v13821_20,v13821_25,v13821_30,v13821_35,v13821_40,v13821_45,v13821_50,v13821_55,v13821_60,v13821_65,v13821_70,v13821_75,v13821_80,v13821_85,v13821_90,v13821_98,v13821_99,v13860,v13861,v9001,v13866,v13868,v13867,v13869,v9025,v13870,v13871,v13876,v13877,v13878,v13884,v13881,v13882,v9010,v9002,v9004_1,v9004_2,v9004_3,v9004_4,v9004_5,v9004_6,v9004_7,v9004_8,v9004_9,v9004_10,v9004_11,v9004_12,v9004_13,v9004_98,v9004_99,v9006_1,v9006_2,v9006_3,v9006_4,v9006_5,v9006_6,v9006_7,v9006_8,v9006_9,v9006_10,v9006_11,v9006_12,v9006_13,v9006_14,v9006_15,v9006_16,v9006_19,v9006_20,v9006_98,v9006_99,v9007_1,v9007_2,v9007_3,v9007_4,v9007_5,v9007_6,v9007_7,v9007_8,v9007_9,v9007_10,v9007_11,v9007_12,v9007_13,v9007_14,v9007_15,v9007_16,v9007_17,v9007_18,v9007_19,v9007_20,v9007_21,v9007_22,v9007_23,v9007_24,v9007_25,v9007_26,v9007_27,v9007_0,v9007_99,v13874,v9011,v13892_2,v13892_4,v13892_6,v13892_8,v13892_10,v13892_12,v13892_98,v13893_10,v13893_15,v13893_20,v13893_25,v13893_30,v13893_35,v13893_40,v13893_45,v13893_50,v13893_55,v13893_60,v13893_65,v13893_70,v13893_75,v13893_80,v13893_85,v13893_90,v13893_98,v9867_1,v9867_2,v9867_3,v9867_4,v9867_98,v9867_97,v9867_0,v9867_99,v13875,v9012,v9013,v9014,v9015,v13865,v9016,v9017,v9018_1,v9018_2,v9018_3,v9018_4,v9018_5,v9018_6,v9018_7,v9018_8,v9018_9,v9018_10,v9018_11,v9018_12,v9018_13,v9018_14,v9018_15,v9018_16,v9018_17,v9018_0,v9018_99,v9027_1,v9027_2,v9027_3,v9027_4,v9027_98,v9027_0,v9027_99,v9032,v9033_1,v9033_2,v9033_3,v9033_4,v9033_5,v9033_6,v9033_7,v9033_8,v9033_9,v9033_10,v9033_11,v9033_12,v9033_13,v9033_14,v9033_15,v9033_16,v9033_17,v9033_18,v9033_19,v9033_20,v9033_98,v9033_99,v13894,v13895,v16372_1,v16372_2,v16372_3,v16372_4,v16372_5,v16372_6,v16372_7,v16372_8,v8976,v13896,v8977,v13901,v13898,v13897,v13916,v8994,v13913,v13903,v13904,v13905,v8982,v8983,v8984,v13918,v8985,v13919,v13935,v13934,v8986,v8987_1,v8987_2,v8987_3,v8987_4,v8987_5,v8987_6,v8987_7,v8987_8,v8987_9,v8987_10,v8987_11,v8987_12,v8987_13,v8987_14,v8987_15,v8987_16,v8987_17,v8987_0,v8987_99,v8996_1,v8996_2,v8996_3,v8996_4,v8996_0,v8996_99,v8980_1,v8980_2,v8980_3,v8980_4,v8980_5,v8980_6,v8980_7,v8980_8,v8980_9,v8980_10,v8980_11,v8980_12,v8980_13,v8980_98,v8980_99,v8978,v8979,v9853_1,v9853_2,v9853_3,v9853_4,v9853_5,v9853_6,v9853_98,v9853_0,v9853_99,v9854,v13920,v13921,v13922,v13902,v14142,v13924,v13937_5,v13937_10,v13937_15,v13937_20,v13937_25,v13937_30,v13937_35,v13937_40,v13938,v9035,v9036_2021,v9036_2020,v9036_2019,v9036_2018,v9036_2017,v9036_2016,v9036_2015,v9036_2014,v9036_2013,v9036_2012,v9036_2011,v9036_2010,v9036_2009,v9036_2008,v9036_2007,v9036_2006,v9036_2005,v9036_2004,v9036_98,v9036_99,v9869,v13939,v13936,v14146,v9873_1,v9873_2,v9873_3,v9873_4,v9873_5,v9873_6,v9873_7,v9873_8,v9873_9,v9873_10,v9873_11,v9873_12,v9873_13,v9873_98,v9873_99,v9879,v9880,v9037,v13946,v13941,v13942_2,v13942_4,v13942_6,v13942_8,v13942_98,v13943,v13945,v9038_1,v9038_2,v9038_3,v9038_4,v9038_5,v9038_6,v9038_7,v9038_8,v9038_9,v9038_10,v9038_11,v9038_12,v9038_13,v9038_14,v9038_15,v9038_16,v9038_17,v9038_18,v9038_19,v9038_20,v9038_21,v9038_98,v9038_99,v14056_1,v14056_2,v14056_3,v14056_4,v14056_5,v14056_6,v14056_7,v14056_8,v14056_9,v14056_10,v14056_11,v14056_12,v14056_13,v14056_14,v14056_15,v14056_16,v14056_17,v14056_18,v14056_19,v14056_20,v14056_21,v14056_98,v9041,v14058,v14057_10,v14057_15,v14057_20,v14057_25,v14057_30,v14057_35,v14057_40,v14057_45,v14057_50,v14057_55,v14057_60,v14057_98,v14057_97,v9040,v13947_5,v13947_10,v13947_15,v13947_20,v14050,v14051,v14052,v9042,v14059_5,v14059_10,v14059_15,v14059_20,v14059_25,v14059_98,v14060,v8970,v8971,v8972,v14063,v14061,v14062_10,v14062_15,v14062_20,v14062_25,v14062_30,v14062_35,v14062_40,v14062_45,v14062_50,v14062_55,v14062_60,v14062_65,v14062_70,v14062_98,v14064,v14065_10,v14065_15,v14065_20,v14065_25,v14065_30,v14065_35,v14065_40,v14065_45,v14065_50,v14065_55,v14065_60,v14065_98,v14065_97,v14066,usuario,data_hora_preenchimento,data_hora_extracao
codigo_municipio,,,,,,,,

In [27]:
df.loc[:, 'v9036_2021':"v9036_99"] = df.loc[:, 'v9036_2021':"v9036_99"].astype('object')

In [28]:
df.loc[:, 'v9007_1':"v9007_99"] = df.loc[:, 'v9007_1':"v9007_99"].astype('object')

In [29]:
df.dtypes

v8973                              object
v13819                             object
v13820                             object
v13821_10                          object
v13821_15                          object
v13821_20                          object
v13821_25                          object
v13821_30                          object
v13821_35                          object
v13821_40                          object
v13821_45                          object
v13821_50                          object
v13821_55                          object
v13821_60                          object
v13821_65                          object
v13821_70                          object
v13821_75                          object
v13821_80                          object
v13821_85                          object
v13821_90                          object
v13821_98                          object
v13821_99                          object
v13860                             object
v13861                            

In [30]:
colunas = df.columns
list(colunas)

['v8973',
 'v13819',
 'v13820',
 'v13821_10',
 'v13821_15',
 'v13821_20',
 'v13821_25',
 'v13821_30',
 'v13821_35',
 'v13821_40',
 'v13821_45',
 'v13821_50',
 'v13821_55',
 'v13821_60',
 'v13821_65',
 'v13821_70',
 'v13821_75',
 'v13821_80',
 'v13821_85',
 'v13821_90',
 'v13821_98',
 'v13821_99',
 'v13860',
 'v13861',
 'v9001',
 'v13866',
 'v13868',
 'v13867',
 'v13869',
 'v9025',
 'v13870',
 'v13871',
 'v13876',
 'v13877',
 'v13878',
 'v13884',
 'v13881',
 'v13882',
 'v9010',
 'v9002',
 'v9004_1',
 'v9004_2',
 'v9004_3',
 'v9004_4',
 'v9004_5',
 'v9004_6',
 'v9004_7',
 'v9004_8',
 'v9004_9',
 'v9004_10',
 'v9004_11',
 'v9004_12',
 'v9004_13',
 'v9004_98',
 'v9004_99',
 'v9006_1',
 'v9006_2',
 'v9006_3',
 'v9006_4',
 'v9006_5',
 'v9006_6',
 'v9006_7',
 'v9006_8',
 'v9006_9',
 'v9006_10',
 'v9006_11',
 'v9006_12',
 'v9006_13',
 'v9006_14',
 'v9006_15',
 'v9006_16',
 'v9006_19',
 'v9006_20',
 'v9006_98',
 'v9006_99',
 'v9007_1',
 'v9007_2',
 'v9007_3',
 'v9007_4',
 'v9007_5',
 'v9007_6',

In [31]:
columns_to_binarize = [
 'v13821_10',
 'v13821_15',
 'v13821_20',
 'v13821_25',
 'v13821_30',
 'v13821_35',
 'v13821_40',
 'v13821_45',
 'v13821_50',
 'v13821_55',
 'v13821_60',
 'v13821_65',
 'v13821_70',
 'v13821_75',
 'v13821_80',
 'v13821_85',
 'v13821_90',
 'v13821_98',
 'v13821_99',
 'v9004_1',
 'v9004_2',
 'v9004_3',
 'v9004_4',
 'v9004_5',
 'v9004_6',
 'v9004_7',
 'v9004_8',
 'v9004_9',
 'v9004_10',
 'v9004_11',
 'v9004_12',
 'v9004_13',
 'v9004_98',
 'v9004_99',
 'v9006_1',
 'v9006_2',
 'v9006_3',
 'v9006_4',
 'v9006_5',
 'v9006_6',
 'v9006_7',
 'v9006_8',
 'v9006_9',
 'v9006_10',
 'v9006_11',
 'v9006_12',
 'v9006_13',
 'v9006_14',
 'v9006_15',
 'v9006_16',
 'v9006_19',
 'v9006_20',
 'v9006_98',
 'v9006_99',
 'v9007_1',
 'v9007_2',
 'v9007_3',
 'v9007_4',
 'v9007_5',
 'v9007_6',
 'v9007_7',
 'v9007_8',
 'v9007_9',
 'v9007_10',
 'v9007_11',
 'v9007_12',
 'v9007_13',
 'v9007_14',
 'v9007_15',
 'v9007_16',
 'v9007_17',
 'v9007_18',
 'v9007_19',
 'v9007_20',
 'v9007_21',
 'v9007_22',
 'v9007_23',
 'v9007_24',
 'v9007_25',
 'v9007_26',
 'v9007_27',
 'v9007_0',
 'v9007_99',
 'v13892_2',
 'v13892_4',
 'v13892_6',
 'v13892_8',
 'v13892_10',
 'v13892_12',
 'v13892_98',
 'v13893_10',
 'v13893_15',
 'v13893_20',
 'v13893_25',
 'v13893_30',
 'v13893_35',
 'v13893_40',
 'v13893_45',
 'v13893_50',
 'v13893_55',
 'v13893_60',
 'v13893_65',
 'v13893_70',
 'v13893_75',
 'v13893_80',
 'v13893_85',
 'v13893_90',
 'v13893_98',
 'v9867_1',
 'v9867_2',
 'v9867_3',
 'v9867_4',
 'v9867_98',
 'v9867_97',
 'v9867_0',
 'v9867_99',
 'v9018_1',
 'v9018_2',
 'v9018_3',
 'v9018_4',
 'v9018_5',
 'v9018_6',
 'v9018_7',
 'v9018_8',
 'v9018_9',
 'v9018_10',
 'v9018_11',
 'v9018_12',
 'v9018_13',
 'v9018_14',
 'v9018_15',
 'v9018_16',
 'v9018_17',
 'v9018_0',
 'v9018_99',
 'v9027_1',
 'v9027_2',
 'v9027_3',
 'v9027_4',
 'v9027_98',
 'v9027_0',
 'v9027_99',
 'v9033_1',
 'v9033_2',
 'v9033_3',
 'v9033_4',
 'v9033_5',
 'v9033_6',
 'v9033_7',
 'v9033_8',
 'v9033_9',
 'v9033_10',
 'v9033_11',
 'v9033_12',
 'v9033_13',
 'v9033_14',
 'v9033_15',
 'v9033_16',
 'v9033_17',
 'v9033_18',
 'v9033_19',
 'v9033_20',
 'v9033_98',
 'v9033_99',
 'v16372_1',
 'v16372_2',
 'v16372_3',
 'v16372_4',
 'v16372_5',
 'v16372_6',
 'v16372_7',
 'v16372_8',
 'v8987_1',
 'v8987_2',
 'v8987_3',
 'v8987_4',
 'v8987_5',
 'v8987_6',
 'v8987_7',
 'v8987_8',
 'v8987_9',
 'v8987_10',
 'v8987_11',
 'v8987_12',
 'v8987_13',
 'v8987_14',
 'v8987_15',
 'v8987_16',
 'v8987_17',
 'v8987_0',
 'v8987_99',
 'v8996_1',
 'v8996_2',
 'v8996_3',
 'v8996_4',
 'v8996_0',
 'v8996_99',
 'v8980_1',
 'v8980_2',
 'v8980_3',
 'v8980_4',
 'v8980_5',
 'v8980_6',
 'v8980_7',
 'v8980_8',
 'v8980_9',
 'v8980_10',
 'v8980_11',
 'v8980_12',
 'v8980_13',
 'v8980_98',
 'v8980_99',
 'v9853_1',
 'v9853_2',
 'v9853_3',
 'v9853_4',
 'v9853_5',
 'v9853_6',
 'v9853_98',
 'v9853_0',
 'v9853_99',
 'v13937_5',
 'v13937_10',
 'v13937_15',
 'v13937_20',
 'v13937_25',
 'v13937_30',
 'v13937_35',
 'v13937_40',
 'v9036_2021',
 'v9036_2020',
 'v9036_2019',
 'v9036_2018',
 'v9036_2017',
 'v9036_2016',
 'v9036_2015',
 'v9036_2014',
 'v9036_2013',
 'v9036_2012',
 'v9036_2011',
 'v9036_2010',
 'v9036_2009',
 'v9036_2008',
 'v9036_2007',
 'v9036_2006',
 'v9036_2005',
 'v9036_2004',
 'v9036_98',
 'v9036_99',
 'v9873_1',
 'v9873_2',
 'v9873_3',
 'v9873_4',
 'v9873_5',
 'v9873_6',
 'v9873_7',
 'v9873_8',
 'v9873_9',
 'v9873_10',
 'v9873_11',
 'v9873_12',
 'v9873_13',
 'v9873_98',
 'v9873_99',
 'v13942_2',
 'v13942_4',
 'v13942_6',
 'v13942_8',
 'v13942_98',
 'v9038_1',
 'v9038_2',
 'v9038_3',
 'v9038_4',
 'v9038_5',
 'v9038_6',
 'v9038_7',
 'v9038_8',
 'v9038_9',
 'v9038_10',
 'v9038_11',
 'v9038_12',
 'v9038_13',
 'v9038_14',
 'v9038_15',
 'v9038_16',
 'v9038_17',
 'v9038_18',
 'v9038_19',
 'v9038_20',
 'v9038_21',
 'v9038_98',
 'v9038_99',
 'v14056_1',
 'v14056_2',
 'v14056_3',
 'v14056_4',
 'v14056_5',
 'v14056_6',
 'v14056_7',
 'v14056_8',
 'v14056_9',
 'v14056_10',
 'v14056_11',
 'v14056_12',
 'v14056_13',
 'v14056_14',
 'v14056_15',
 'v14056_16',
 'v14056_17',
 'v14056_18',
 'v14056_19',
 'v14056_20',
 'v14056_21',
 'v14056_98',
 'v14057_10',
 'v14057_15',
 'v14057_20',
 'v14057_25',
 'v14057_30',
 'v14057_35',
 'v14057_40',
 'v14057_45',
 'v14057_50',
 'v14057_55',
 'v14057_60',
 'v14057_98',
 'v14057_97',
 'v13947_5',
 'v13947_10',
 'v13947_15',
 'v13947_20',
 'v14059_5',
 'v14059_10',
 'v14059_15',
 'v14059_20',
 'v14059_25',
 'v14059_98',
 'v14062_10',
 'v14062_15',
 'v14062_20',
 'v14062_25',
 'v14062_30',
 'v14062_35',
 'v14062_40',
 'v14062_45',
 'v14062_50',
 'v14062_55',
 'v14062_60',
 'v14062_65',
 'v14062_70',
 'v14062_98',
 'v14065_10',
 'v14065_15',
 'v14065_20',
 'v14065_25',
 'v14065_30',
 'v14065_35',
 'v14065_40',
 'v14065_45',
 'v14065_50',
 'v14065_55',
 'v14065_60',
 'v14065_98',
 'v14065_97'
]

In [32]:
df[columns_to_binarize] = df[columns_to_binarize].astype(bool).astype(int)
df.head()

,v8973,v13819,v13820,v13821_10,v13821_15,v13821_20,v13821_25,v13821_30,v13821_35,v13821_40,v13821_45,v13821_50,v13821_55,v13821_60,v13821_65,v13821_70,v13821_75,v13821_80,v13821_85,v13821_90,v13821_98,v13821_99,v13860,v13861,v9001,v13866,v13868,v13867,v13869,v9025,v13870,v13871,v13876,v13877,v13878,v13884,v13881,v13882,v9010,v9002,v9004_1,v9004_2,v9004_3,v9004_4,v9004_5,v9004_6,v9004_7,v9004_8,v9004_9,v9004_10,v9004_11,v9004_12,v9004_13,v9004_98,v9004_99,v9006_1,v9006_2,v9006_3,v9006_4,v9006_5,v9006_6,v9006_7,v9006_8,v9006_9,v9006_10,v9006_11,v9006_12,v9006_13,v9006_14,v9006_15,v9006_16,v9006_19,v9006_20,v9006_98,v9006_99,v9007_1,v9007_2,v9007_3,v9007_4,v9007_5,v9007_6,v9007_7,v9007_8,v9007_9,v9007_10,v9007_11,v9007_12,v9007_13,v9007_14,v9007_15,v9007_16,v9007_17,v9007_18,v9007_19,v9007_20,v9007_21,v9007_22,v9007_23,v9007_24,v9007_25,v9007_26,v9007_27,v9007_0,v9007_99,v13874,v9011,v13892_2,v13892_4,v13892_6,v13892_8,v13892_10,v13892_12,v13892_98,v13893_10,v13893_15,v13893_20,v13893_25,v13893_30,v13893_35,v13893_40,v13893_45,v13893_50,v13893_55,v13893_60,v13893_65,v13893_70,v13893_75,v13893_80,v13893_85,v13893_90,v13893_98,v9867_1,v9867_2,v9867_3,v9867_4,v9867_98,v9867_97,v9867_0,v9867_99,v13875,v9012,v9013,v9014,v9015,v13865,v9016,v9017,v9018_1,v9018_2,v9018_3,v9018_4,v9018_5,v9018_6,v9018_7,v9018_8,v9018_9,v9018_10,v9018_11,v9018_12,v9018_13,v9018_14,v9018_15,v9018_16,v9018_17,v9018_0,v9018_99,v9027_1,v9027_2,v9027_3,v9027_4,v9027_98,v9027_0,v9027_99,v9032,v9033_1,v9033_2,v9033_3,v9033_4,v9033_5,v9033_6,v9033_7,v9033_8,v9033_9,v9033_10,v9033_11,v9033_12,v9033_13,v9033_14,v9033_15,v9033_16,v9033_17,v9033_18,v9033_19,v9033_20,v9033_98,v9033_99,v13894,v13895,v16372_1,v16372_2,v16372_3,v16372_4,v16372_5,v16372_6,v16372_7,v16372_8,v8976,v13896,v8977,v13901,v13898,v13897,v13916,v8994,v13913,v13903,v13904,v13905,v8982,v8983,v8984,v13918,v8985,v13919,v13935,v13934,v8986,v8987_1,v8987_2,v8987_3,v8987_4,v8987_5,v8987_6,v8987_7,v8987_8,v8987_9,v8987_10,v8987_11,v8987_12,v8987_13,v8987_14,v8987_15,v8987_16,v8987_17,v8987_0,v8987_99,v8996_1,v8996_2,v8996_3,v8996_4,v8996_0,v8996_99,v8980_1,v8980_2,v8980_3,v8980_4,v8980_5,v8980_6,v8980_7,v8980_8,v8980_9,v8980_10,v8980_11,v8980_12,v8980_13,v8980_98,v8980_99,v8978,v8979,v9853_1,v9853_2,v9853_3,v9853_4,v9853_5,v9853_6,v9853_98,v9853_0,v9853_99,v9854,v13920,v13921,v13922,v13902,v14142,v13924,v13937_5,v13937_10,v13937_15,v13937_20,v13937_25,v13937_30,v13937_35,v13937_40,v13938,v9035,v9036_2021,v9036_2020,v9036_2019,v9036_2018,v9036_2017,v9036_2016,v9036_2015,v9036_2014,v9036_2013,v9036_2012,v9036_2011,v9036_2010,v9036_2009,v9036_2008,v9036_2007,v9036_2006,v9036_2005,v9036_2004,v9036_98,v9036_99,v9869,v13939,v13936,v14146,v9873_1,v9873_2,v9873_3,v9873_4,v9873_5,v9873_6,v9873_7,v9873_8,v9873_9,v9873_10,v9873_11,v9873_12,v9873_13,v9873_98,v9873_99,v9879,v9880,v9037,v13946,v13941,v13942_2,v13942_4,v13942_6,v13942_8,v13942_98,v13943,v13945,v9038_1,v9038_2,v9038_3,v9038_4,v9038_5,v9038_6,v9038_7,v9038_8,v9038_9,v9038_10,v9038_11,v9038_12,v9038_13,v9038_14,v9038_15,v9038_16,v9038_17,v9038_18,v9038_19,v9038_20,v9038_21,v9038_98,v9038_99,v14056_1,v14056_2,v14056_3,v14056_4,v14056_5,v14056_6,v14056_7,v14056_8,v14056_9,v14056_10,v14056_11,v14056_12,v14056_13,v14056_14,v14056_15,v14056_16,v14056_17,v14056_18,v14056_19,v14056_20,v14056_21,v14056_98,v9041,v14058,v14057_10,v14057_15,v14057_20,v14057_25,v14057_30,v14057_35,v14057_40,v14057_45,v14057_50,v14057_55,v14057_60,v14057_98,v14057_97,v9040,v13947_5,v13947_10,v13947_15,v13947_20,v14050,v14051,v14052,v9042,v14059_5,v14059_10,v14059_15,v14059_20,v14059_25,v14059_98,v14060,v8970,v8971,v8972,v14063,v14061,v14062_10,v14062_15,v14062_20,v14062_25,v14062_30,v14062_35,v14062_40,v14062_45,v14062_50,v14062_55,v14062_60,v14062_65,v14062_70,v14062_98,v14064,v14065_10,v14065_15,v14065_20,v14065_25,v14065_30,v14065_35,v14065_40,v14065_45,v14065_50,v14065_55,v14065_60,v14065_98,v14065_97,v14066,usuario,data_hora_preenchimento,data_hora_extracao
codigo_municipio,,,,,,,,

In [33]:
df.dtypes

v8973                              object
v13819                             object
v13820                             object
v13821_10                           int32
v13821_15                           int32
v13821_20                           int32
v13821_25                           int32
v13821_30                           int32
v13821_35                           int32
v13821_40                           int32
v13821_45                           int32
v13821_50                           int32
v13821_55                           int32
v13821_60                           int32
v13821_65                           int32
v13821_70                           int32
v13821_75                           int32
v13821_80                           int32
v13821_85                           int32
v13821_90                           int32
v13821_98                           int32
v13821_99                           int32
v13860                             object
v13861                            

In [34]:
df['v14050'] = df['v14050'].astype('object')

In [35]:
v8973_dict =  {
    'Gerência de Segurança Alimentar e Nutricional (ou Divisão, Superintendência ou similar)': 1, 
    'Coordenação de Segurança Alimentar e Nutricional (ou Seção, Setor ou similar)': 2, 
    'Departamento de Segurança Alimentar e Nutricional (ou Diretoria, Secretaria Executiva, Secretaria Adjunta, Subsecretaria ou similar)': 3,
    'Secretaria Municipal de Segurança Alimentar e Nutricional ou similar': 4, 
    'Gabinete do Prefeito': 5, 
    'Câmara Intersetorial de Segurança Alimentar e Nutricional - Caisan': 6, 
    'Outro órgão municipal': 98,
    'Não há um órgão para coordenar as açőes de Segurança Alimentar e Nutricional': 0 
}

In [36]:
df['v8973'].keys()

Float64Index([1200336.0, 1200807.0, 2700300.0, 2701357.0, 2701407.0, 2702801.0, 2703403.0, 2704005.0, 2705002.0, 2705101.0,
              ...
              1720978.0, 1721109.0, 1721208.0, 1721257.0, 1721307.0, 1722081.0, 1722107.0,       nan,       nan,       nan], dtype='float64', name='codigo_municipio', length=1938)

In [37]:
df['v8973'] = df['v8973'].map(v8973_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v8973'] = df['v8973'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v8973'].value_counts()

 98    690
 0     508
 6     200
-1     151
 2     137
 4      85
 3      78
 5      62
 1      27
Name: v8973, dtype: int64

In [38]:
v8973_dict

{'Gerência de Segurança Alimentar e Nutricional (ou Divisão, Superintendência ou similar)': 1,
 'Coordenação de Segurança Alimentar e Nutricional (ou Seção, Setor ou similar)': 2,
 'Departamento de Segurança Alimentar e Nutricional (ou Diretoria, Secretaria Executiva, Secretaria Adjunta, Subsecretaria ou similar)': 3,
 'Secretaria Municipal de Segurança Alimentar e Nutricional ou similar': 4,
 'Gabinete do Prefeito': 5,
 'Câmara Intersetorial de Segurança Alimentar e Nutricional - Caisan': 6,
 'Outro órgão municipal': 98,
 'Não há um órgão para coordenar as açőes de Segurança Alimentar e Nutricional': 0}

In [39]:
atrib_lbl('v8973_dict', 'labels.sps', dicionario)

Arquivo "labels.sps" atualizado com sucesso.


In [40]:
df.v8973.value_counts()

 98    690
 0     508
 6     200
-1     151
 2     137
 4      85
 3      78
 5      62
 1      27
Name: v8973, dtype: int64

In [41]:
df['v13819'].unique()


array(['Outro órgão municipal', 'Outra Secretaria Municipal',
       'Secretaria Municipal de Assistência Social / Desenvolvimento Social ou órgão similar',
       'Secretaria Municipal de Desenvolvimento Agrário / Agricultura / Abastecimento ou órgão similar',
       'Secretaria Municipal de Saúde ou órgão similar',
       'Gabinete do Executivo / Prefeito(a)',
       'Secretaria Municipal de Educação ou órgão similar',
       'Secretaria Municipal do Trabalho / Empreendedorismo / Desenvolvimento Econômico ou órgão similar',
       'Secretaria Municipal de Esporte, Lazer, Juventude ou órgão similar',
       'Secretaria Municipal de Direitos Humanos ou órgão similar', nan],
      dtype=object)

In [42]:
v13819_dict = {
'Gabinete do Executivo / Prefeito(a)': 9,
'Secretaria Municipal de Assistência Social / Desenvolvimento Social ou órgão similar': 8,
'Secretaria Municipal de Desenvolvimento Agrário / Agricultura / Abastecimento ou órgão similar': 7,
'Secretaria Municipal do Trabalho / Empreendedorismo / Desenvolvimento Econômico ou órgão similar': 6,
'Secretaria Municipal de Educação ou órgão similar': 5,
'Secretaria Municipal de Saúde ou órgão similar': 4,
'Secretaria Municipal de Direitos Humanos ou órgão similar': 3,
'Secretaria Municipal de Cultura ou órgão similar': 2,
'Secretaria Municipal de Esporte, Lazer, Juventude ou órgão similar': 1,
'Outra Secretaria Municipal ou Outro órgão municipal': 98
}

atrib_lbl('v13819_dict', 'labels.sps', dicionario)

Arquivo "labels.sps" atualizado com sucesso.


In [43]:
df['v13819'] = df['v13819'].map(v13819_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13819'] = df['v13819'].map(lambda x: int(x) if not np.isnan(x) else -1)



df['v13819'].value_counts()

 8    1035
-1     305
 7     241
 5     179
 4      83
 9      76
 6      13
 3       4
 1       2
Name: v13819, dtype: int64

In [44]:
df[df.columns.difference(columns_to_binarize)].columns

Index(['data_hora_extracao', 'data_hora_preenchimento', 'usuario', 'v13819', 'v13820', 'v13860', 'v13861', 'v13865', 'v13866', 'v13867', 'v13868', 'v13869', 'v13870', 'v13871', 'v13874', 'v13875', 'v13876', 'v13877', 'v13878', 'v13881', 'v13882', 'v13884', 'v13894', 'v13895', 'v13896', 'v13897', 'v13898', 'v13901', 'v13902', 'v13903', 'v13904', 'v13905', 'v13913', 'v13916', 'v13918', 'v13919', 'v13920', 'v13921', 'v13922', 'v13924', 'v13934', 'v13935', 'v13936', 'v13938', 'v13939', 'v13941', 'v13943', 'v13945', 'v13946', 'v14050', 'v14051', 'v14052', 'v14058', 'v14060', 'v14061', 'v14063', 'v14064', 'v14066', 'v14142', 'v14146', 'v8970', 'v8971', 'v8972', 'v8973', 'v8976', 'v8977', 'v8978', 'v8979', 'v8982', 'v8983', 'v8984', 'v8985', 'v8986', 'v8994', 'v9001', 'v9002', 'v9010', 'v9011', 'v9012', 'v9013', 'v9014', 'v9015', 'v9016', 'v9017', 'v9025', 'v9032', 'v9035', 'v9037', 'v9040', 'v9041', 'v9042', 'v9854', 'v9869', 'v9879', 'v9880'], dtype='object')

In [45]:
df['v13820'].unique()

array(['Não Sabe',
       'Órgão que coordena açőes de SAN realizadas em apenas uma secretaria ou área de política pública',
       'Nenhuma das opçőes anteriores',
       'Órgão de articulação intersetorial, que coordena açőes de SAN realizadas em diversas secretarias / setores de políticas públicas',
       nan], dtype=object)

In [46]:
v13820_dict = {
'Órgão de articulação intersetorial, que coordena açőes de SAN realizadas em diversas secretarias / setores de políticas públicas': 3,  
'Órgão que coordena açőes de SAN realizadas em apenas uma secretaria ou área de política pública': 2,
'Nenhuma das opçőes anteriores': 97,
'Não Sabe': 99
}

atrib_lbl('v13820_dict', 'labels.sps', dicionario)

df['v13820'] = df['v13820'].map(v13820_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13820'] = df['v13820'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13820'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


 3     871
 2     407
 97    355
 99    302
-1       3
Name: v13820, dtype: int64

In [47]:
df.loc[df['v13820'] == -1, 'v13820'] = 99

In [48]:
df['v13860'].unique()

array(['Não sabe',
       'Não, o município ainda não aderiu ao Sisan, mas tem interesse em aderir em dois anos ou mais',
       'Não, o município ainda não aderiu ao Sisan, mas tem interesse em aderir em até um ano',
       'Sim, o município aderiu formalmente ao Sisan',
       'Sim, o município está em meio ao processo de adesão ao Sisan',
       'Sim, o município já deu iniciou ao trâmite de adesão ao Sisan junto ŕ Caisan Estadual e ao Consea Estadual',
       'Não, o município não aderiu ao Sisan, e não pretende aderir futuramente',
       nan], dtype=object)

In [49]:
v13860_dict = {
    'Sim, o município aderiu formalmente ao Sisan': 10,
    'Sim, o município está em meio ao processo de adesão ao Sisan': 8,
    'Sim, o município já deu iniciou ao trâmite de adesão ao Sisan junto ŕ Caisan Estadual e ao Consea Estadual': 6,
    'Não, o município ainda não aderiu ao Sisan, mas tem interesse em aderir em dois anos ou mais': 4,
    'Não, o município ainda não aderiu ao Sisan, mas tem interesse em aderir em até um ano': 2,
    'Não, o município não aderiu ao Sisan, e não pretende aderir futuramente': 0,
    'Não sabe': 99
}

df['v13860'] = df['v13860'].map(v13860_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13860'] = df['v13860'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13860'].value_counts()

 10    475
 4     463
 2     438
 99    355
 8      89
 6      61
 0      54
-1       3
Name: v13860, dtype: int64

In [50]:
atrib_lbl('v13860_dict', 'labels.sps', dicionario)

Arquivo "labels.sps" atualizado com sucesso.


In [51]:
df['v13861'].unique()

array([nan, 'Não Sabe',
       'Sim, muito interesse por parte de gestores municipais',
       'Sim, algum interesse por parte de alguns gestores municipais',
       'Sim, pouco interesse por parte de gestores municipais',
       'Não tem nenhum interesse em aderir ao Sisan no futuro'],
      dtype=object)

In [52]:
df['v13861'].isna().sum()

984

In [53]:
v13861_dict = {
       'Sim, muito interesse por parte de gestores municipais': 10,
       'Sim, algum interesse por parte de alguns gestores municipais': 6,
       'Sim, pouco interesse por parte de gestores municipais': 2,
       'Não tem nenhum interesse em aderir ao Sisan no futuro': 0,
       'Não Sabe': 99
    }

atrib_lbl('v13861_dict', 'labels.sps', dicionario)

df['v13861'] = df['v13861'].map(v13861_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13861'] = df['v13861'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13861'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     984
 10    598
 6     271
 99     43
 2      36
 0       6
Name: v13861, dtype: int64

In [54]:
list(df['v13877'].unique())

[nan,
 'De 2 a 3 anos',
 'Não Sabe',
 'Há menos de 1 ano',
 'De 4 a 5 anos',
 'Há mais de 8 anos',
 'De 6 a 7 anos']

In [55]:
v13877_dict = {
    'Há menos de 1 ano': 1,
    'De 2 a 3 anos': 3,
    'De 4 a 5 anos': 5,
    'De 6 a 7 anos': 7,
    'Há mais de 8 anos': 9,
    'Não Sabe': 99
    }

atrib_lbl('v13877_dict', 'labels.sps', dicionario)

df['v13877'] = df['v13877'].map(v13877_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13877'] = df['v13877'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13877'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1041
 1      382
 3      274
 99     115
 5       86
 7       25
 9       15
Name: v13877, dtype: int64

In [56]:
list(df['v13894'].unique())

[nan, 'Não', 'Não Sabe', 'Sim']

In [57]:
v13894_dict = {'Sim': 1, 'Não': 0, 'Não Sabe': 99}

df['v13894'] = df['v13894'].map(v13894_dict)

atrib_lbl('v13894_dict', 'labels.sps', dicionario)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13894'] = df['v13894'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13894'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1046
 1      369
 0      351
 99     172
Name: v13894, dtype: int64

In [58]:
list(df['v13895'].unique())

[nan,
 'Mensal',
 'Bimestral',
 'Não existe um fluxo de comunicação com o Consea Estadual',
 'Semestral',
 'Anual',
 'Trimestral',
 'Não Sabe',
 0]

In [59]:
df['v13895'].value_counts()

Mensal                                                      113
Bimestral                                                    56
Não existe um fluxo de comunicação com o Consea Estadual     55
Semestral                                                    51
Trimestral                                                   40
Anual                                                        38
Não Sabe                                                     16
0                                                             1
Name: v13895, dtype: int64

In [60]:
v13895_dict = {
    'Mensal': 1,
    'Bimestral': 2,
    'Trimestral': 3,
    'Semestral': 6,
    'Anual': 12,
    'Não existe um fluxo de comunicação com o Consea Estadual': 0,
    'Não Sabe': 99,
    }


atrib_lbl('v13895_dict', 'labels.sps', dicionario)

df['v13895'] = df['v13895'].map(v13895_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13895'] = df['v13895'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13895'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1569
 1      113
 2       56
 0       55
 6       51
 3       40
 12      38
 99      16
Name: v13895, dtype: int64

In [61]:
df['v13895'].value_counts()

-1     1569
 1      113
 2       56
 0       55
 6       51
 3       40
 12      38
 99      16
Name: v13895, dtype: int64

In [62]:
list(df['v13895'].unique())

[-1, 1, 2, 0, 6, 12, 3, 99]

In [63]:
list(df['v13904'].unique())

[nan,
 'Não Sabe',
 'Há menos de 1 ano',
 'De 2 a 3 anos',
 'De 4 a 5 anos',
 'De 6 a 7 anos',
 'Há mais de 8 anos']

In [64]:
df['v13904'].value_counts()

Há menos de 1 ano    182
De 2 a 3 anos        115
Não Sabe              64
De 4 a 5 anos         54
De 6 a 7 anos         10
Há mais de 8 anos      3
Name: v13904, dtype: int64

In [65]:
v13904_dict = {
    'Há menos de 1 ano': 1,
    'De 2 a 3 anos': 3,
    'De 4 a 5 anos': 5,
    'De 6 a 7 anos': 7,
    'Há mais de 8 anos': 9,
    'Não há!': 0,
    'Não Sabe': 99
    }

atrib_lbl('v13904_dict', 'labels.sps', dicionario)

df['v13904'] = df['v13904'].map(v13904_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13904'] = df['v13904'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13904'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1510
 1      182
 3      115
 99      64
 5       54
 7       10
 9        3
Name: v13904, dtype: int64

In [66]:
list(df['v13919'].unique())

[nan,
 'Menos de 1 ano',
 '1 a 2 anos',
 '5 a 6 anos',
 'Não Sabe',
 'Mais de 8 anos',
 '3 a 4 anos',
 '7 a 8 anos',
 0]

In [67]:
v13919_dict = {
    'Menos de 1 ano': 1,
    '1 a 2 anos': 2,
    '3 a 4 anos': 4,
    '5 a 6 anos': 6,
    '7 a 8 anos': 8,
    'Mais de 8 anos': 9,
    'Não Sabe': 99
    }


atrib_lbl('v13919_dict', 'labels.sps', dicionario)

df['v13919'] = df['v13919'].map(v13919_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13919'] = df['v13919'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13919'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1739
 1       71
 2       41
 4       31
 6       27
 8       13
 9       11
 99       5
Name: v13919, dtype: int64

In [68]:
df['v13919'].value_counts()

-1     1739
 1       71
 2       41
 4       31
 6       27
 8       13
 9       11
 99       5
Name: v13919, dtype: int64

In [69]:
list(df['v13924'].unique())

[nan,
 'Mensal',
 'Quinzenal',
 'Anual',
 'Trimestral',
 'Bimestral',
 'Semestral',
 'Não existe um fluxo de comunicação com a Caisan Estadual',
 'Não Sabe',
 0]

In [70]:
v13924_dict = {
 'Quinzenal': 1,
 'Mensal': 2,
 'Bimestral': 3,
 'Trimestral': 4,
 'Semestral': 5,
 'Anual': 6,
 'Não existe um fluxo de comunicação com a Caisan Estadual': 0,
 'Não Sabe': 99,
}


atrib_lbl('v13924_dict', 'labels.sps', dicionario)

df['v13924'] = df['v13924'].map(v13924_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13924'] = df['v13924'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13924'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1805
 2       25
 3       23
 0       22
 5       21
 6       18
 4       13
 1        6
 99       5
Name: v13924, dtype: int64

In [71]:
list(df['v13935'].unique())

[nan,
 'Menos de 1 ano',
 '3 a 4 anos',
 '1 a 2 anos',
 '5 a 6 anos',
 'Mais de 8 anos',
 '7 a 8 anos',
 'Não Sabe',
 0]

In [72]:
v13935_dict = {
    'Menos de 1 ano': 1,
    '1 a 2 anos': 2,
    '3 a 4 anos': 4,
    '5 a 6 anos': 6,
    '7 a 8 anos': 8,
    'Mais de 8 anos': 9,
    'Não Sabe': 99,
    0: -1
    }

atrib_lbl('v13935_dict', 'labels.sps', dicionario)

df['v13935'] = df['v13935'].map(v13935_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13935'] = df['v13935'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13935'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1739
 1       70
 2       42
 4       32
 6       23
 9       18
 8       10
 99       4
Name: v13935, dtype: int64

In [73]:
# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13936'] = df['v13936'].map(lambda x: int(x) if not np.isnan(x) else -1)
df['v13936'].value_counts()

-1       1316
 20        49
 10        41
 30        36
 15        32
 6         26
 5         25
 8         23
 12        22
 4         16
 11        15
 40        14
 35        13
 25        13
 16        12
 32        12
 13        11
 14        11
 23        11
 18        11
 0         10
 7         10
 2          9
 43         8
 27         8
 21         8
 24         8
 60         8
 22         8
 31         7
 28         7
 3          7
 48         7
 50         6
 1          6
 9          6
 39         6
 42         5
 33         5
 19         5
 26         5
 53         4
 45         4
 17         4
 65         4
 49         4
 36         4
 37         3
 70         3
 75         3
 80         3
 34         3
 47         3
 52         2
 90         2
 41         2
 61         2
 38         2
 54         1
 58         1
 71         1
 79         1
 57         1
 84         1
 1532       1
 59         1
 88         1
 270        1
 85         1
 200        1
 144        1
 81   

In [74]:
list(df['v13939'].unique())

[nan, 'Sim', 'Não Sabe', 'Não']

In [75]:
v13939_dict = {'Sim': 1, 'Não': 0, 'Não Sabe': 99}

atrib_lbl('v13939_dict', 'labels.sps', dicionario)

df['v13939'] = df['v13939'].map(v13939_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13939'] = df['v13939'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13939'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1281
 1      622
 99      31
 0        4
Name: v13939, dtype: int64

In [76]:
list(df['v13941'].unique())

[nan, 'Sim', 'Não', 'Não Sabe', 0]

In [77]:
sn_dict(df, 'v13941')

Arquivo "labels.sps" atualizado com sucesso.
Dicionário sim/não/não_sabe concluído. Frequências:


(None,
 -1     1759
  1      138
  0       27
  99      14
 Name: v13941, dtype: int64)

In [78]:
list(df['v13946'].unique())

[nan,
 'Não foi realizado estudo ou diagnóstico específico com a finalidade de subsidiar a elaboração do Plano Municipal de SAN, mas foram utilizados dados sobre a situação de SAN disponíveis, bases de dados municipais e outras fontes de informação',
 'Não Sabe',
 'Sim, um diagnóstico atualizado da situação da SAN no município foi realizado especificamente para fundamentar a elaboração do Plano municipal de SAN',
 'O Plano Municipal foi elaborado com base em diagnóstico sobre a situação da SAN realizado anteriormente (pré-existente e anterior ao ano de elaboração do Plamsan)',
 'Não foi realizado disgnóstico ou estudo da situação de SAN no município para subsidiar a elaboração do Plano municipal de SAN',
 0]

In [79]:
autodict(df, 'v13946')

{nan: 0,
 'Não foi realizado estudo ou diagnóstico específico com a finalidade de subsidiar a elaboração do Plano Municipal de SAN, mas foram utilizados dados sobre a situação de SAN disponíveis, bases de dados municipais e outras fontes de informação': 1,
 'Não Sabe': 2,
 'Sim, um diagnóstico atualizado da situação da SAN no município foi realizado especificamente para fundamentar a elaboração do Plano municipal de SAN': 3,
 'O Plano Municipal foi elaborado com base em diagnóstico sobre a situação da SAN realizado anteriormente (pré-existente e anterior ao ano de elaboração do Plamsan)': 4,
 'Não foi realizado disgnóstico ou estudo da situação de SAN no município para subsidiar a elaboração do Plano municipal de SAN': 5,
 0: 6}

In [80]:
v13946_dict = {
 'Sim, um diagnóstico atualizado da situação da SAN no município foi realizado especificamente para fundamentar a elaboração do Plano municipal de SAN': 5,
 'O Plano Municipal foi elaborado com base em diagnóstico sobre a situação da SAN realizado anteriormente (pré-existente e anterior ao ano de elaboração do Plamsan)': 4,
 'Não foi realizado estudo ou diagnóstico específico com a finalidade de subsidiar a elaboração do Plano Municipal de SAN, mas foram utilizados dados sobre a situação de SAN disponíveis, bases de dados municipais e outras fontes de informação': 3,
 'Não foi realizado disgnóstico ou estudo da situação de SAN no município para subsidiar a elaboração do Plano municipal de SAN': 2,
 'Não Sabe': 99 
}

df['v13946'] = df['v13946'].map(v13946_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v13946'] = df['v13946'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v13946'].value_counts()

-1     1759
 5       96
 3       36
 4       33
 99      11
 2        3
Name: v13946, dtype: int64

In [81]:
atrib_lbl('v13946_dict', 'labels.sps', dicionario)

Arquivo "labels.sps" atualizado com sucesso.


In [82]:
autodict(df, 'v14052')

{nan: 0,
 'Terceiro ano': 1,
 'Não Sabe': 2,
 'Segundo ano': 3,
 'Quarto ou último ano de vigência': 4,
 'Primeiro ano': 5,
 0: 6}

In [83]:
v14052_dict = {
 'Primeiro ano': 1,
 'Segundo ano': 2,
 'Terceiro ano': 3,
 'Quarto ou último ano de vigência': 4,
 'Não Sabe': 99 
}

df['v14052'] = df['v14052'].map(v14052_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v14052'] = df['v14052'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v14052'].value_counts()

-1     1814
 4       44
 99      28
 2       27
 1       13
 3       12
Name: v14052, dtype: int64

In [84]:
atrib_lbl('v14052_dict', 'labels.sps', dicionario)

Arquivo "labels.sps" atualizado com sucesso.


In [85]:
autodict(df, 'v14058')

{nan: 0,
 'Sim, contemplou parte das propostas deliberadas na Conferência Municipal de SAN': 1,
 'Sim, contemplou todas as propostas da Conferência Municipal de SAN': 2,
 'Não Sabe': 3,
 'O município não ainda não realizou uma Conferência Municipal de SAN': 4,
 'Sim, contemplou apenas algumas propostas, que estavam em conssonância com a agenda do governo municipal': 5,
 0: 6,
 'Não contemplou nenhuma das propostas deliberadas na Conferência Municipal de SAN': 7}

In [86]:
v14058_dict = {
 'Sim, contemplou todas as propostas da Conferência Municipal de SAN': 4,
 'Sim, contemplou apenas algumas propostas, que estavam em conssonância com a agenda do governo municipal': 3,
 'Sim, contemplou parte das propostas deliberadas na Conferência Municipal de SAN': 2,
 'Não contemplou nenhuma das propostas deliberadas na Conferência Municipal de SAN': 1, 
 'O município não ainda não realizou uma Conferência Municipal de SAN': 0,
 'Não Sabe': 99 
}

df['v14058'] = df['v14058'].map(v14058_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v14058'] = df['v14058'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v14058'].value_counts()

-1     1759
 4       95
 2       50
 99      19
 3       10
 0        4
 1        1
Name: v14058, dtype: int64

In [87]:
atrib_lbl('v14058_dict', 'labels.sps', dicionario)

Arquivo "labels.sps" atualizado com sucesso.


In [88]:
autodict(df, 'v14060')

{nan: 0, 'Sim': 1, 'Não': 2, 'Não Sabe': 3}

In [89]:
sn_dict(df, 'v14060')

Arquivo "labels.sps" atualizado com sucesso.
Dicionário sim/não/não_sabe concluído. Frequências:


(None,
 -1     1759
  1       73
  0       65
  99      41
 Name: v14060, dtype: int64)

In [90]:
autodict(df, 'v14061')

{nan: 0, 'Sim': 1, 'Não Sabe': 2, 'Não': 3}

In [91]:
sn_dict(df, 'v14061')

Arquivo "labels.sps" atualizado com sucesso.
Dicionário sim/não/não_sabe concluído. Frequências:


(None,
 -1     1323
  1      503
  99      67
  0       45
 Name: v14061, dtype: int64)

In [92]:
autodict(df, 'v8970')

{'Não Sabe': 0, 'Não': 1, 'Sim': 2, nan: 3}

In [93]:
sn_dict(df, 'v8970')

Arquivo "labels.sps" atualizado com sucesso.
Dicionário sim/não/não_sabe concluído. Frequências:


(None,
  0     886
  1     619
  99    426
 -1       7
 Name: v8970, dtype: int64)

In [94]:
df.loc[df['v8970'] == -1, 'v8970'] = 99

In [95]:
sn_dict(df, 'v8976')

Arquivo "labels.sps" atualizado com sucesso.
Dicionário sim/não/não_sabe concluído. Frequências:


(None,
  0     1219
  1      431
  99     282
 -1        6
 Name: v8976, dtype: int64)

In [96]:
df.loc[df['v8976'] == -1, 'v8976'] = 99


In [97]:
autodict(df, 'v8978')

{nan: 0,
 'Não sabe': 1,
 'Nunca fez reuniőes': 2,
 'Há mais de um ano': 3,
 'Entre três meses e seis meses': 4,
 'Há menos de três meses': 5,
 'Entre seis meses e um ano': 6,
 'Há menos de um mês': 7}

In [98]:
v8978_dict = {
    'Há menos de um mês': 12,
    'Há menos de três meses': 5,
    'Entre três meses e seis meses': 3,
    'Entre seis meses e um ano': 2,
    'Há mais de um ano': 1,
    'Nunca fez reuniőes': 0,
    'Não sabe': 99
    }

atrib_lbl('v8978_dict', 'labels.sps', dicionario)

df['v8978'] = df['v8978'].map(v8978_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v8978'] = df['v8978'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v8978'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1510
 1       87
 99      68
 12      65
 5       63
 0       54
 2       53
 3       38
Name: v8978, dtype: int64

In [99]:
autodict(df, 'v8979')

{nan: 0,
 'Não sabe': 1,
 'Mensal': 2,
 'Não há periodicidade': 3,
 'Bimestral': 4,
 'Trimestral': 5,
 'Semestral': 6,
 0: 7,
 'Anual': 8}

In [100]:
v8979_dict = {
    'Mensal': 12,
    'Bimestral': 6,
    'Trimestral': 4,
    'Semestral': 2,
    'Anual': 1,
    'Não há periodicidade': 0,
    'Não sabe': 99    
    }


atrib_lbl('v8979_dict', 'labels.sps', dicionario)

df['v8979'] = df['v8979'].map(v8979_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v8979'] = df['v8979'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v8979'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1510
 0      128
 99      69
 6       59
 4       59
 2       54
 12      44
 1       15
Name: v8979, dtype: int64

In [101]:
autodict(df, 'v8982')

{nan: 0,
 'Não Sabe': 1,
 'Secretaria de Assistência Social (Ação Social, Promoção Social, Desenvolvimento Social ou similar)': 2,
 'Secretaria de Saúde (ou similar)': 3,
 'Outro órgão municipal': 4,
 'Secretaria de Agricultura ou Agropecuária (ou similar)': 5,
 'Secretaria de Segurança Alimentar e Nutricional ou similar': 6,
 'Gabinete do Prefeito': 7,
 'Secretaria de Educação (ou similar)': 8,
 'Não há vínculo com nenhum outro órgão municipal': 9,
 'Secretaria de Desenvolvimento Agrário ou Rural (ou similar)': 10}

In [102]:
v8982_dict = {
'Gabinete do Prefeito': 12,
'Secretaria de Segurança Alimentar e Nutricional ou similar': 10,
'Secretaria de Assistência Social (Ação Social, Promoção Social, Desenvolvimento Social ou similar)': 8,
'Secretaria de Desenvolvimento Agrário ou Rural (ou similar)': 6,
'Secretaria de Agricultura ou Agropecuária (ou similar)': 4,
'Secretaria de Saúde (ou similar)': 3,
'Secretaria de Educação (ou similar)': 2,
'Não Sabe': 99,
'Outro órgão municipal': 98,
'Não há vínculo com nenhum outro órgão municipal': 0  
}


atrib_lbl('v8982_dict', 'labels.sps', dicionario)

df['v8982'] = df['v8982'].map(v8982_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v8982'] = df['v8982'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v8982'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1510
 8      223
 4       85
 2       46
 12      32
 99      19
 3        7
 98       7
 10       4
 0        4
 6        1
Name: v8982, dtype: int64

In [103]:
autodict(df, 'v8983')

{nan: 0, 'Não sabe': 1, 'Não': 2, 'Sim, mas não sabe quanto': 3, 'Sim': 4}

In [104]:
v8983_dict = {
    'Sim': 1,
    'Sim, mas não sabe quanto': 2,
    'Não': 0,
    'Não sabe': 99
    }


atrib_lbl('v8983_dict', 'labels.sps', dicionario)

df['v8983'] = df['v8983'].map(v8983_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v8983'] = df['v8983'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v8983'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1510
 0      291
 99      80
 2       36
 1       21
Name: v8983, dtype: int64

In [105]:
autodict(df, 'v8985')

{nan: 0, 'Não Sabe': 1, 'Sim': 2, 'Não': 3}

In [106]:
sn_dict(df, 'v8985')

Arquivo "labels.sps" atualizado com sucesso.
Dicionário sim/não/não_sabe concluído. Frequências:


(None,
 -1     1510
  1      199
  0      189
  99      40
 Name: v8985, dtype: int64)

In [107]:
autodict(df, 'v8986')

{nan: 0,
 0.0: 1,
 4.0: 2,
 1.0: 3,
 2.0: 4,
 8.0: 5,
 3.0: 6,
 5.0: 7,
 12.0: 8,
 6.0: 9,
 7.0: 10}

In [108]:
# aplicando a função map para converter a série em inteiros e tratar NaN
df['v8986'] = df['v8986'].map(lambda x: int(x) if not np.isnan(x) else -1)

In [109]:
df['v8986'].value_counts()

-1     1509
 0      175
 1      167
 2       50
 3       19
 4        8
 5        4
 8        2
 6        2
 12       1
 7        1
Name: v8986, dtype: int64

In [110]:
autodict(df, 'v9001')

{'Não Sabe': 0, 'Não': 1, 'Sim': 2, nan: 3}

In [111]:
sn_dict(df, 'v9001')

Arquivo "labels.sps" atualizado com sucesso.
Dicionário sim/não/não_sabe concluído. Frequências:


(None,
  1     900
  0     863
  99    172
 -1       3
 Name: v9001, dtype: int64)

In [112]:
df.loc[df['v9001'] == -1, 'v9001'] = 99
df['v9001'].value_counts()

1     900
0     863
99    175
Name: v9001, dtype: int64

In [113]:
autodict(df, 'v9002')

{nan: 0,
 '1/3 governo e 2/3 sociedade civil': 1,
 'Não Sabe': 2,
 'Paritário (metade de governo e metade da sociedade civil)': 3,
 'Outra Proporção': 4,
 'Maioria de representantes de governo': 5}

In [114]:
v9002_dict = {
 '1/3 governo e 2/3 sociedade civil': 3,
 'Paritário (metade de governo e metade da sociedade civil)': 2,
 'Maioria de representantes de governo': 1,
 'Outra Proporção': 98,
 'Não Sabe': 99
 }


atrib_lbl('v9002_dict', 'labels.sps', dicionario)

df['v9002'] = df['v9002'].map(v9002_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v9002'] = df['v9002'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v9002'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1044
 3      680
 2      158
 98      28
 99      24
 1        4
Name: v9002, dtype: int64

In [115]:
autodict(df, 'v9010')

{nan: 0,
 'Secretaria de Assistência Social (Ação Social, Promoção Social, Desenvolvimento Social ou similar)': 1,
 'Gabinete do Prefeito': 2,
 'Não há vínculo com nenhum outro órgão municipal': 3,
 'Secretaria de Agricultura ou Agropecuária (ou similar)': 4,
 'Não Sabe': 5,
 'Outro órgão municipal': 6,
 'Secretaria de Saúde (ou similar)': 7,
 'Secretaria de Segurança Alimentar e Nutricional ou similar': 8,
 'Secretaria de Educação (ou similar)': 9,
 'Secretaria de Desenvolvimento Agrário ou Rural (ou similar)': 10,
 'Abastecimento': 11}

In [116]:
v9010_dict = {
'Gabinete do Prefeito': 16,
'Secretaria de Segurança Alimentar e Nutricional ou similar': 14,
'Secretaria de Assistência Social (Ação Social, Promoção Social,Desenvolvimento Social ou similar)': 12,
'Secretaria de Desenvolvimento Agrário ou Rural (ou similar)': 10,
'Secretaria de Agricultura ou Agropecuária (ou similar)': 8,
'Abastecimento': 6,
'Secretaria de Saúde (ou similar)': 4,
'Secretaria de Educação (ou similar)': 2,
'Outro órgão municipal': 98,
'Não há vínculo com nenhum outro órgão municipal': 0,
'Não Sabe': 99
 }


atrib_lbl('v9010_dict', 'labels.sps', dicionario)

df['v9010'] = df['v9010'].map(v9010_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v9010'] = df['v9010'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v9010'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1609
 8      128
 2       71
 16      46
 98      25
 99      18
 0       16
 4       13
 10       7
 14       4
 6        1
Name: v9010, dtype: int64

In [119]:
autodict(df, 'v9011')

{nan: 0,
 'Consultivo ? tem papel de propor e recomendar sobre as açőes de SAN ŕ gestão municipal': 1,
 'Não Sabe': 2,
 'Consultivo e Deliberativo': 3,
 'Deliberativo ? tem poder de decisão para determinar quais devem ser as açőes da política de SAN na gestão municipal': 4}

In [120]:
v9011_dict = {
'Consultivo ? tem papel de propor e recomendar sobre as açőes de SAN ŕ gestão municipal': 3,
'Deliberativo ? tem poder de decisão para determinar quais devem ser as açőes da política de SAN na gestão municipal': 2,
'Consultivo e Deliberativo': 1,
'Não Sabe': 99  
}

atrib_lbl('v9011_dict', 'labels.sps', dicionario)

df['v9011'] = df['v9011'].map(v9011_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v9011'] = df['v9011'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v9011'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1045
 1      448
 3      309
 2       87
 99      49
Name: v9011, dtype: int64

In [121]:
autodict(df, 'v9012')

{nan: 0,
 'Mensal': 1,
 'Não Sabe': 2,
 'Bimestral': 3,
 'Outra': 4,
 'Anual': 5,
 'Semestral': 6}

In [122]:
v9012_dict = {
    'Mensal': 12,
    'Bimestral': 6,
    'Semestral': 2,
    'Anual': 1,
    'Outra':98,
    'Não sabe': 99    
    }

atrib_lbl('v9012_dict', 'labels.sps', dicionario)

df['v9012'] = df['v9012'].map(v9012_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v9012'] = df['v9012'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v9012'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1161
 12     319
 6      203
 98     123
 2       91
 1       41
Name: v9012, dtype: int64

In [123]:
autodict(df, 'v9013')

{nan: 0,
 'No último mês': 1,
 'Não Sabe': 2,
 'Nunca fez reuniőes': 3,
 'Entre seis meses e um ano': 4,
 'Nos últimos três meses': 5,
 'Há mais de um ano': 6,
 'Entre três meses e seis meses': 7}

In [124]:
v9013_dict = {
'No último mês': 12,
'Nos últimos três meses': 5,
'Entre três meses e seis meses': 3, 
'Entre seis meses e um ano': 2,
'Há mais de um ano': 1,
'Nunca fez reuniőes': 0,
'Não Sabe': 99
}

atrib_lbl('v9013_dict', 'labels.sps', dicionario)

df['v9013'] = df['v9013'].map(v9013_dict)

# aplicando a função map para converter a série em inteiros e tratar NaN
df['v9013'] = df['v9013'].map(lambda x: int(x) if not np.isnan(x) else -1)

df['v9013'].value_counts()

Arquivo "labels.sps" atualizado com sucesso.


-1     1044
 12     309
 1      188
 5      137
 99      89
 3       77
 2       72
 0       22
Name: v9013, dtype: int64